In [4]:
#Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score
sns.set()
#read necessary csv files
users=pd.read_csv('takehome_users.csv',encoding='latin-1')
engagement=pd.read_csv('takehome_user_engagement.csv',encoding='latin-1')
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [10]:
engagement.head()
#change timestamp to date time
engagement['time_stamp']=pd.to_datetime(engagement['time_stamp'],infer_datetime_format=True)
engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [26]:
temp=pd.pivot_table(engagement, values='visited', index=['user_id', 'time_stamp'])
temp.head()

visited
user_id time_stamp                  
1       2014-04-22 03:53:30        1
2       2013-11-15 03:45:04        1
        2013-11-29 03:45:04        1
        2013-12-09 03:45:04        1
        2013-12-25 03:45:04        1

In [27]:
#getting largest number of logins in per week for each user through reformating
#resampling of multiindex for every 2 weeks
level_values = temp.index.get_level_values
t2 = (temp.groupby([level_values(i) for i in [0,1]] +[pd.Grouper(freq='1w', level=-1)])).count()
t2=t2.reset_index(level=1)
t2=t2.drop(['time_stamp'],axis=1)
t2=t2.reset_index()
#group by user id and timestamp(max)
t3=t2.groupby(['user_id','time_stamp']).sum()
t3=t3.reset_index()
t3=t3.groupby('user_id').max()
final=t3.drop('time_stamp',axis=1)
final=final.reset_index()



In [28]:
full=users.merge(final, how='left',  left_on='object_id', right_on='user_id')
full.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id,visited
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,1.0,1.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2.0,3.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,3.0,1.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,4.0,1.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,5.0,1.0


In [34]:
full['adopted']=full.visited>=3 #At least 3 visits means adoption
full=full.drop(['object_id','creation_time','name','email','last_session_creation_time','visited','invited_by_user_id'],axis=1)
#drop data that is not of interest (nothing to do with adoption)
full.head() #we can now use T/F for adopted or not

,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted
0,GUEST_INVITE,1,0,11,False
1,ORG_INVITE,0,0,1,True
2,ORG_INVITE,0,0,94,False
3,GUEST_INVITE,0,0,1,False
4,GUEST_INVITE,0,0,193,False


In [35]:
#Create X and y train sets so we can run regression
y=full['adopted'].values
datax=full.drop(['adopted'],axis=1)
datax=pd.get_dummies(datax,drop_first=False)
X=datax.values
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)

In [36]:
#Run logistic regression
logreg=LogisticRegression()
param_grid = {'C':[0.01,0.1,1,10,100]}
logreg_cv = GridSearchCV(logreg,param_grid,cv=5)

logreg_cv.fit(X_train,y_train)
predicted=logreg_cv.predict(X_test)
score=logreg_cv.score(X_train,y_train)
print(score)

0.8795238095238095


In [37]:
#Fit coefficients to factors and view the table
logreg.fit(X_train, y_train)
coef=logreg.coef_
columns=datax.columns

data_coef=pd.DataFrame(data=coef[0],index=columns)
data_ranked=data_coef.sort_values(0)
data_ranked

,0
creation_source_PERSONAL_PROJECTS,-0.900256
creation_source_ORG_INVITE,-0.398354
creation_source_SIGNUP,-0.256974
creation_source_GUEST_INVITE,-0.146968
creation_source_SIGNUP_GOOGLE_AUTH,-0.146739
org_id,0.001390
opted_in_to_mailing_list,0.040606
enabled_for_marketing_drip,0.060018


Creation source seems to be the strongest factor for predicting if a user will adopt. With Org_id, opted_in_to_mailing_list, and enabled_for_marketing_drip having 0 to no effect on user adoption.

In [44]:
full.head()
creation=full.groupby('creation_source').mean()
creation

,opted_in_to_mailing_list,org_id,adopted
enabled_for_marketing_drip,,,
0,0.161834,141.402527,0.119612
1,0.748884,144.630580,0.125000


In [45]:
opted=full.groupby('opted_in_to_mailing_list').mean()
opted

,enabled_for_marketing_drip,org_id,adopted
opted_in_to_mailing_list,,,
0,0.049967,141.639129,0.118477
1,0.448230,142.622912,0.126253


In [46]:
enabled=full.groupby('enabled_for_marketing_drip').mean()
enabled

,opted_in_to_mailing_list,org_id,adopted
enabled_for_marketing_drip,,,
0,0.161834,141.402527,0.119612
1,0.748884,144.630580,0.125000


From the above data we can see that the creation_source has the most powerful impact on user adoption. The average value for the other columns. Creation source is therefore the best predictor with guest_invite being the best positive predictor.